In [1]:
import os
import json
import time
import random
from pathlib import Path
from typing import List, Dict, Any, Tuple, Callable, Optional
from tqdm import tqdm

# --- Configuration ---

# Directory where 'sample.ipynb' saved its data
DATA_ROOT = Path("knapsack_multisize_data")

# Directory to save the output of this notebook
RESULTS_DIR = Path("experiment_results")
RESULTS_DIR.mkdir(exist_ok=True)

# Protocol defined in team_project_plan.md
N_RUNS_RANDOMIZED = 3

# A base seed for the solver *runs* (distinct from instance generation seeds)
BASE_RUN_SEED = 20241019 

print(f"Data root (input): {DATA_ROOT.resolve()}")
print(f"Results dir (output): {RESULTS_DIR.resolve()}")

Data root (input): C:\Users\abhay\Desktop\Projects\COMA_IIITR\knapsack_multisize_data
Results dir (output): C:\Users\abhay\Desktop\Projects\COMA_IIITR\experiment_results


In [2]:
def load_instance_json(path: Path) -> Optional[Dict]:
    "Helper to load a single JSON instance file."
    if not path.exists():
        print(f"Warning: File not found, skipping: {path}")
        return None
    try:
        with path.open("r", encoding="utf8") as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading {path}: {e}")
        return None

def load_all_instances(data_root: Path) -> List[Tuple[str, str, str, Dict, Path]]:
    """
    Loads all 27 instances based on the metadata file.
    
    Returns:
        A flat list of tuples, where each tuple is:
        (n, dist_name, cap_name, instance_dict, instance_path)
    """
    meta_path = data_root / "generation_metadata.json"
    if not meta_path.exists():
        print(f"Error: Metadata file not found at {meta_path}")
        print("Please run 'sample.ipynb' first to generate the datasets.")
        return []
    
    with meta_path.open("r", encoding="utf8") as f:
        meta = json.load(f)
        
    all_instances = []
    print(f"Loading instances from {meta_path}...")

    # meta[dist_name][cap_name] -> list of records
    for dist_name, cap_dict in meta.items():
        for cap_name, records in cap_dict.items():
            for rec in records:
                n = rec["n"]
                json_path = Path(rec["json"])
                
                inst_data = load_instance_json(json_path)
                
                if inst_data:
                    all_instances.append((n, dist_name, cap_name, inst_data, json_path))
    
    print(f"Successfully loaded {len(all_instances)} instances.")
    return all_instances

# --- Load the data ---
# This list contains all 27 problem instances
ALL_INSTANCES = load_all_instances(DATA_ROOT)

if ALL_INSTANCES:
    print("\nExample instance check:")
    n, dist, cap, inst, path = ALL_INSTANCES[0]
    print(f"  n = {n}, dist = {dist}, cap = {cap}")
    print(f"  Path = {path.name}")
    print(f"  Capacity = {inst['capacity']}")
    print(f"  Num Items = {len(inst['items'])}")

Loading instances from knapsack_multisize_data\generation_metadata.json...
Successfully loaded 27 instances.

Example instance check:
  n = 100, dist = uniform, cap = cap_0.2
  Path = knapsack_n100_seed20251020.json
  Capacity = 5049
  Num Items = 100


In [3]:
# --- Solver Interface (Type Hint) ---
SolverFunction = Callable[[Dict, Optional[float], Optional[int]], Tuple[List[int], float, Dict]]

# --- Import your solvers from external .py files ---
# Ensure my_solvers.py is in the same directory as this notebook
try:
    from my_solvers import (
        quantum_annealing_solver
    )
    
    print("Successfully imported all 15 custom solvers from my_solvers.py")
    
except ImportError as e:
    print(f"WARNING: Could not import from my_solvers.py. {e}")
    print("Please create 'my_solvers.py' in the same directory.")
    print("Using inline dummy solvers only.")
    
    # Define placeholder functions if import fails so notebook doesn't crash
    def placeholder_solver(*args, **kwargs): 
        raise NotImplementedError("Solver not found. Check my_solvers.py.")
    
    quantum_annealing_solver = placeholder_solver

# --- ** DEFINE THE FULL LIST OF SOLVERS TO RUN ** ---
SOLVERS_TO_RUN = {
    "QuantumAnnealing": {
        "function": quantum_annealing_solver,
        "is_randomized": True,
        "params": {
            "num_reads": 500,
            "penalty_multiplier": 10,
        }
    }
}

print(f"Defined {len(SOLVERS_TO_RUN)} solvers to run.")
print(f"Solvers: {list(SOLVERS_TO_RUN.keys())}")

Successfully imported all 15 custom solvers from my_solvers.py
Defined 1 solvers to run.
Solvers: ['QuantumAnnealing']


In [4]:
# Create a flat list of all tasks to run
tasks = []
for solver_name, config in SOLVERS_TO_RUN.items():
    # (n, dist_name, cap_name, instance_dict, instance_path)
    for (n, dist, cap, instance, path) in ALL_INSTANCES:
        
        n_runs = N_RUNS_RANDOMIZED if config["is_randomized"] else 1
        
        for i in range(n_runs):
            # Each randomized run gets a unique, reproducible seed
            run_seed = (BASE_RUN_SEED + i) if config["is_randomized"] else None
            tasks.append((solver_name, config, n, dist, cap, instance, path, run_seed, i, n_runs))

print(f"Total experiment tasks to run: {len(tasks)}")

all_results = []
start_total_time = time.perf_counter()

Total experiment tasks to run: 81


In [5]:
all_results = []
start_total_time = time.perf_counter()

for task in tqdm(tasks, desc="Running Solvers"):
    (solver_name, config, n, dist, cap, instance, path, run_seed, run_idx, n_runs) = task

    solver_func = config["function"]
    solver_params = config.get("params", {}) or {}

    try:
        # --- Execute the solver and time it ---
        start_run_time = time.perf_counter()

        # Forward solver-specific params using kwargs.
        # This keeps signature: solver_func(instance=..., timeout=..., seed=..., **solver_params)
        best_solution_ids, best_value, logs = solver_func(
            instance=instance,
            timeout=None,          # or set a timeout per your policy
            seed=run_seed,
            **solver_params
        )

        end_run_time = time.perf_counter()
        runtime = end_run_time - start_run_time
        # ----------------------------------------

        # Build the result record
        result_record = {
            "method": solver_name,
            "parameters": {**config.get("params", {})},  # show params used
            "seed": run_seed,
            "run_index": run_idx,
            "instance_file": str(path.name) if path is not None else None,
            "instance_n": n,
            "instance_dist": dist,
            "instance_cap_ratio_str": cap,
            "instance_seed": instance.get('meta', {}).get('seed') if isinstance(instance, dict) else None,
            "best_value": best_value,
            "runtime": runtime,
            "logs": logs
        }

        all_results.append(result_record)

    except Exception as e:
        print(f"!! ERROR running {solver_name} on {path.name if path is not None else 'unknown'} (Run {run_idx+1}) !!")
        print(f"  Error: {e}")
        all_results.append({
            "method": solver_name,
            "parameters": config.get("params", {}),
            "seed": run_seed,
            "instance_file": str(path.name) if path is not None else None,
            "best_value": None,
            "runtime": None,
            "logs": {"error": str(e)}
        })

end_total_time = time.perf_counter()
print(f"\n--- Execution Complete ---")
print(f"Total results collected: {len(all_results)}")
print(f"Total time: {end_total_time - start_total_time:.2f} seconds")

Running Solvers: 100%|██████████| 81/81 [20:34<00:00, 15.24s/it]


--- Execution Complete ---
Total results collected: 81
Total time: 1234.44 seconds


In [6]:
output_path = RESULTS_DIR / "quantum_solver_runs.json"

try:
    with open(output_path, "w", encoding="utf8") as f:
        json.dump(all_results, f, indent=2)
    
    print(f"Successfully saved {len(all_results)} results to:")
    print(f"{output_path.resolve()}")

except TypeError as e:
    print(f"Error: Could not serialize results to JSON. {e}")
    print("Check that your 'logs' dictionary contains JSON-compatible types (no numpy arrays, etc.)")
except Exception as e:
    print(f"An unexpected error occurred while saving: {e}")

Successfully saved 81 results to:
C:\Users\abhay\Desktop\Projects\COMA_IIITR\experiment_results\quantum_solver_runs.json
